In [1]:
import pystac_client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt

from pystac.extensions.eo import EOExtension as eo

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
bbox_of_interest = [-8.4197, 32.6656, -8.4122, 32.6703]
time_of_interest = "2021-10-08/2022-05-05"

In [10]:
query = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={
        "eo:cloud_cover": {"lt": 10},
        "platform": {"in": ["landsat-8", "landsat-9"]},
    },
)
datalist=list(query.get_items())

items = query.item_collection()
print(f"Returned {len(items)} Items")

Returned 32 Items


In [14]:
from odc.stac import configure_rio, stac_load
data = stac_load(
    datalist,
    crs="epsg:32629",
    bands=["lwir11"],
    resolution=30,
    chunks={},
    groupby="solar_day",
    bbox=bbox_of_interest,
)
#data

<xarray.Dataset>
Dimensions:      (y: 19, x: 24, time: 23)
Coordinates:
  * y            (y) float64 3.615e+06 3.615e+06 ... 3.614e+06 3.614e+06
  * x            (x) float64 5.544e+05 5.544e+05 ... 5.551e+05 5.551e+05
    spatial_ref  int32 32629
  * time         (time) datetime64[ns] 2021-10-09T11:10:20.083903 ... 2022-05...
Data variables:
    lwir11       (time, y, x) uint16 dask.array<chunksize=(1, 19, 24), meta=np.ndarray>

<b>Landsat characteristics</b>
- unit: kelvin<br>
- scale: 0.00341802<br>
- nodata: 0<br>
- offset: 149.0<br>
- data type: uint16<br>
- spatial resolution: 30m<br>

In [18]:
example = data.isel(time=0).lwir11
height, width = example.shape

In [28]:
import numpy as np
import rasterio

# Metadata
metadata = {
    'driver': 'GTiff',
    'dtype': rasterio.float32,
    'count': 1,
    'height': height,
    'width': width,
    'transform': rasterio.Affine(30, 0, 554400, 0, -30, 3614910),  # Transformation par défaut pour une grille régulière
    'crs': rasterio.crs.CRS.from_epsg(32629)  # Système de coordonnées de référence (par exemple, EPSG:4326 pour WGS84)
}
# LST correction with offset and scale and data export
for i in range(data.coords["time"].size):
    arr = str(np.array(data.coords["time"][i]))[:-19]
    
    array_lst = data.isel(time=i).lwir11
    corr = np.float32(array_lst)
    array_lst = corr
    
    # correction
    array_lst *= 0.00341802
    array_lst += 149.0
    array_lst -= 273.15 # Kelvin to Celsius
    
    # Write
    output_file = "/home/eoafrica/shared/Final Workflow/LST/Sidi_Bennour_LST_{0}.tif".format(arr)
    #print("LST: ",arr,"........100%")
    # Creation of TIFF file
    with rasterio.open(output_file, 'w', **metadata) as dst:
        dst.write(array_lst, 1)  # Writing in band 1
        
    print("+++ Date ",arr," is complete +++")

LST:  2021-10-09 ........100%
LST:  2021-10-18 ........100%
LST:  2021-11-07 ........100%
LST:  2021-11-12 ........100%
LST:  2021-11-22 ........100%
LST:  2021-12-04 ........100%
LST:  2021-12-05 ........100%
LST:  2021-12-12 ........100%
LST:  2021-12-13 ........100%
LST:  2021-12-29 ........100%
LST:  2022-01-05 ........100%
LST:  2022-01-06 ........100%
LST:  2022-01-13 ........100%
LST:  2022-01-14 ........100%
LST:  2022-01-21 ........100%
LST:  2022-01-30 ........100%
LST:  2022-02-07 ........100%
LST:  2022-02-15 ........100%
LST:  2022-02-23 ........100%
LST:  2022-03-03 ........100%
LST:  2022-03-19 ........100%
LST:  2022-04-28 ........100%
LST:  2022-05-05 ........100%
